## Librerias

In [ ]:
#Importo las librerias que voy a utilizar para EDA

import numpy as np 
import pandas as pd 
import cv2
import matplotlib
import matplotlib.pyplot as plt
import random
import seaborn as sns
import os
%matplotlib inline

## Directorio

In [ ]:
path = '/kaggle/input/vinbigdata-chest-xray-abnormalities-detection/'
os.listdir(path)

## **Lectura del CSV**

In [ ]:
#Realizo la lectura del dataset

df = pd.read_csv(path+"train.csv")

## Caracteristicas generales

In [ ]:
#Observo las primeras filas del DataFrame

df.head()

In [ ]:
#Observo los tipos de datos por columna

df.dtypes

In [ ]:
#Observo el tamaño del dataframe

df.shape

In [ ]:
#Listo las columnas del dataframe

df.columns

In [ ]:
#Observo la cantidad de registros para la columna 'class_name', la cual corresponde al nombre de la clase del objeto detectado

cantidad_por_clase = df['class_name'].value_counts()
cantidad_por_clase

Segun se indica en la descripcion de los datos la relacion entre el nombre de la clase y el id de la clase es la siguiente:

0 - Aortic enlargement

1 - Atelectasis

2 - Calcification

3 - Cardiomegaly

4 - Consolidation

5 - ILD

6 - Infiltration

7 - Lung Opacity

8 - Nodule/Mass

9 - Other lesion

10 - Pleural effusion

11 - Pleural thickening

12 - Pneumothorax

13 - Pulmonary fibrosis


Es importante destacar que el valor 14 corresponde a la ausencia de las observaciones enumeradas anteriormente

In [ ]:
#Realizo el grafico correspondiente

plt.figure(figsize = (8,6))

plt.barh(cantidad_por_clase.index,cantidad_por_clase.values)
plt.title("Cantidad de observaciones por tipo", fontsize=16)
plt.xlabel("Cantidad", fontsize=12)
plt.ylabel("Tipo", fontsize=12)

In [ ]:
#Para ver los valores en % normalizo los datos y multiplico por 100

cantidad_por_clase_normalizado = df['class_name'].value_counts(normalize=True)*100
cantidad_por_clase_normalizado

Se observa que en el 46% de las observaciones no se encuentran anomalias

In [ ]:
#A continuacion analizo los NaN por columna

df.isnull().sum()

Segun se observa los valores nulos se presentan en las columnas donde se incluyen los valores de las coordenadas.


In [ ]:
#Verifico si los NaN corresponden a la Id de la Clase 14, que segun vimos corresponde a la ausencia de observaciones

#Los valores coinciden

df[df['class_id']==14].isnull().sum()

In [ ]:
#Observo la cantidad de registros para la columna 'rad_id', la cual corresponde al id del readiologo que realizo el diagnostico

cantidad_por_radiologo = df['rad_id'].value_counts()
cantidad_por_radiologo

In [ ]:
plt.figure(figsize = (8,6))

plt.barh(cantidad_por_radiologo.index,cantidad_por_radiologo.values,color='r')

plt.title("Cantidad de observaciones por radiologo", fontsize=16)
plt.xlabel("Cantidad", fontsize=12)
plt.ylabel("Radiologo", fontsize=12)

## Caracteristicas de las Clases

In [ ]:
#Defino una funcion para realizar diferentes pairplot de las clases

def plot_pairplot(i):
    plt.figure(figsize=(10, 10))
    sns.pairplot(df[df['class_id']==i],hue='class_id',dropna=True,corner=True)
    plt.show()

A continuacion realizo el pairplot para las 3 clases con mayores observaciones:

    * Aortic enlargement (clas_id=0)

    * Cardiomegaly (clas_id=3)

    * Pleural thickening (clas_id=11)

In [ ]:
#Pairplot para la clase 0

plot_pairplot(0)

In [ ]:
#Pairplot para la clase 1

plot_pairplot(3)

In [ ]:
#Pairplot para la clase 2

plot_pairplot(11)

## Visualizacion de Imagenes DICOM

Segun se observa el formato de las imagenes que se utilizaran para el entranamiento del modelo tienen formato DICOM (Digital Imaging and Communication On Medicine), el cual es un estándar de transmisión de imágenes médicas y datos entre hardware de propósito médico

In [ ]:
#Importo los modulos que voy a utilizar

import pydicom as dicom #pydicom para la lectura de las imagenes DICOM
import cv2

A continuacion realizo la lectura de una imagen del dataset de entrenamiento

In [ ]:
idnum = 3 #Indice del array que voy a utilizar para seleccionar la imagen
 
image_id = df.loc[idnum, 'image_id'] #Selecciono el id de la imagen correspondiente a esa posicion en el DataFrame

data_file = dicom.dcmread(path+'train/'+image_id+'.dicom') #Realizo la lectura de la imagen


In [ ]:
#Obtengo el array de esa imagen

img = data_file.pixel_array
img

In [ ]:
#Observo la metadata de la imagen

print(data_file)

In [ ]:
#Observo el tamaño de la imagen. El mismo tambien se observa en la metadatada

print('Image shape:', img.shape)

In [ ]:
#Realizo la impresion de la imagen con un rectangulo señalando la zona donde se encuentra la patologia

bbox = [df.loc[idnum, 'x_min'],
        df.loc[idnum, 'y_min'],
        df.loc[idnum, 'x_max'],
        df.loc[idnum, 'y_max']]
fig, ax = plt.subplots(1, 1, figsize=(20, 4))
ax.imshow(img, cmap='gray')
p = matplotlib.patches.Rectangle((bbox[0], bbox[1]),
                                 bbox[2]-bbox[0],
                                 bbox[3]-bbox[1],
                                 ec='r', fc='none', lw=2.)
ax.add_patch(p)
plt.show()